In [1]:
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc, sys
import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
train_lines = codecs.open('data/Train_DataSet.csv', 'r', "utf-8").readlines()[1:]
train_df = pd.DataFrame({
            'id': [x[:32] for x in train_lines],
            'ocr': [x[33:].strip() for x in train_lines]
})
train_label = pd.read_csv('data/Train_DataSet_Label.csv')
train_df = pd.merge(train_df, train_label, on='id')

test_lines = codecs.open('data/Test_DataSet.csv', 'r', "utf-8").readlines()[1:]
test_df = pd.DataFrame({
            'id': [x[:32] for x in test_lines],
            'ocr': [x[33:].strip() for x in test_lines]
})

In [3]:
# #! -*- coding:utf-8 -*-
# import re, os, json, codecs, gc
# import numpy as np
# import pandas as pd
# from random import choice
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import KFold
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer

# from keras.layers import *
# from keras.callbacks import *
# from keras.models import Model
# import keras.backend as K
# from keras.optimizers import Adam

maxlen =  510  # 512
config_path = 'F:/EXE/python/bert/chinese_wwm_ext_L-12_H-768_A-12/bert_config.json'
# checkpoint_path = '/export/home/liuyuzhong/kaggle/bert/chinese_L-12_H-768_A-12/bert_model.ckpt'
checkpoint_path = 'F:/EXE/python/bert/chinese_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'F:/EXE/python/bert/chinese_wwm_ext_L-12_H-768_A-12/vocab.txt'

token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

class data_generator:
    def __init__(self, data, batch_size=4, shuffle=True):    # 8
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            
            if self.shuffle:
                np.random.shuffle(idxs)
            
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y[:, 0, :]
                    [X1, X2, Y] = [], [], []

from keras.metrics import top_k_categorical_accuracy
def acc_top2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)
                    
def build_bert(nclass):
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

    for l in bert_model.layers:
        l.trainable = True

    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))

    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x)
    p = Dense(nclass, activation='softmax')(x)

    model = Model([x1_in, x2_in], p)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(1e-5),
                  metrics=['accuracy', acc_top2])
    print(model.summary())
    return model


In [4]:
from keras.utils import to_categorical

DATA_LIST = []
for data_row in train_df.iloc[:].itertuples():
    DATA_LIST.append((data_row.ocr, to_categorical(data_row.label, 3)))
DATA_LIST = np.array(DATA_LIST)

DATA_LIST_TEST = []
for data_row in test_df.iloc[:].itertuples():
    DATA_LIST_TEST.append((data_row.ocr, to_categorical(0, 3)))
DATA_LIST_TEST = np.array(DATA_LIST_TEST)

In [5]:
def run_cv(nfold, data, data_label, data_test):
    kf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=1904).split(data, train_df['label'])
    train_model_pred = np.zeros((len(data), 3))
    test_model_pred = np.zeros((len(data_test), 3))

    for i, (train_fold, test_fold) in enumerate(kf):
        X_train, X_valid, = data[train_fold, :], data[test_fold, :]
        
        model = build_bert(3)
        early_stopping = EarlyStopping(monitor='val_acc', patience=3)
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
        checkpoint = ModelCheckpoint('./model/' + str(i) + '.hdf5', monitor='val_acc', 
                                         verbose=2, save_best_only=True, mode='max',save_weights_only=True)
        
        train_D = data_generator(X_train, shuffle=True)
        valid_D = data_generator(X_valid, shuffle=True)
        test_D = data_generator(data_test, shuffle=False)
        
        model.fit_generator(
            train_D.__iter__(),
            steps_per_epoch=len(train_D),
            epochs=5,
            validation_data=valid_D.__iter__(),
            validation_steps=len(valid_D),
            callbacks=[early_stopping, plateau, checkpoint],
            verbose=2
        )
        
        # model.load_weights('./model/' + str(i) + '.hdf5')
        
        # return model
        train_model_pred[test_fold, :] =  model.predict_generator(valid_D.__iter__(), steps=len(valid_D),verbose=0)
        test_model_pred += model.predict_generator(test_D.__iter__(), steps=len(test_D),verbose=0)
        
        del model; gc.collect()
        K.clear_session()
        
        # break
        
    return train_model_pred, test_model_pred

In [6]:
train_model_pred, test_model_pred = run_cv(5, DATA_LIST, None, DATA_LIST_TEST)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________

In [7]:
test_pred = [np.argmax(x) for x in test_model_pred]
test_df['label'] = test_pred

In [8]:
test_model_pred.shape

(7356, 3)

In [8]:
test_df[['id', 'label']].to_csv('sub_2.csv', index=None)

In [9]:
np.save('bert_test_maxlen510_wwm_test.npy', test_model_pred)

In [ ]:
"""
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 577s - loss: 0.5036 - acc: 0.7930 - acc_top2: 0.9721 - val_loss: 0.3560 - val_acc: 0.8682 - val_acc_top2: 0.9912

Epoch 00001: val_acc improved from -inf to 0.86821, saving model to ./model/0.hdf5
Epoch 2/5
 - 562s - loss: 0.3042 - acc: 0.8792 - acc_top2: 0.9905 - val_loss: 0.3449 - val_acc: 0.8621 - val_acc_top2: 0.9905

Epoch 00002: val_acc did not improve from 0.86821
Epoch 3/5
 - 563s - loss: 0.1658 - acc: 0.9342 - acc_top2: 0.9973 - val_loss: 0.4522 - val_acc: 0.8519 - val_acc_top2: 0.9871

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00003: val_acc did not improve from 0.86821
Epoch 4/5
 - 564s - loss: 0.0607 - acc: 0.9794 - acc_top2: 0.9995 - val_loss: 0.6007 - val_acc: 0.8499 - val_acc_top2: 0.9878

Epoch 00004: val_acc did not improve from 0.86821
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 577s - loss: 0.4998 - acc: 0.7944 - acc_top2: 0.9747 - val_loss: 0.4423 - val_acc: 0.8213 - val_acc_top2: 0.9837

Epoch 00001: val_acc improved from -inf to 0.82133, saving model to ./model/1.hdf5
Epoch 2/5
 - 563s - loss: 0.3019 - acc: 0.8812 - acc_top2: 0.9900 - val_loss: 0.3702 - val_acc: 0.8438 - val_acc_top2: 0.9898

Epoch 00002: val_acc improved from 0.82133 to 0.84375, saving model to ./model/1.hdf5
Epoch 3/5
 - 562s - loss: 0.1749 - acc: 0.9342 - acc_top2: 0.9975 - val_loss: 0.5063 - val_acc: 0.8342 - val_acc_top2: 0.9789

Epoch 00003: val_acc did not improve from 0.84375
Epoch 4/5
 - 562s - loss: 0.0968 - acc: 0.9653 - acc_top2: 0.9988 - val_loss: 0.5990 - val_acc: 0.8288 - val_acc_top2: 0.9837

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00004: val_acc did not improve from 0.84375
Epoch 5/5
 - 562s - loss: 0.0347 - acc: 0.9891 - acc_top2: 0.9997 - val_loss: 0.7666 - val_acc: 0.8431 - val_acc_top2: 0.9830

Epoch 00005: val_acc did not improve from 0.84375
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 578s - loss: 0.4932 - acc: 0.7993 - acc_top2: 0.9730 - val_loss: 0.3995 - val_acc: 0.8334 - val_acc_top2: 0.9864

Epoch 00001: val_acc improved from -inf to 0.83345, saving model to ./model/2.hdf5
Epoch 2/5
 - 565s - loss: 0.2837 - acc: 0.8873 - acc_top2: 0.9920 - val_loss: 0.3904 - val_acc: 0.8498 - val_acc_top2: 0.9878

Epoch 00002: val_acc improved from 0.83345 to 0.84976, saving model to ./model/2.hdf5
Epoch 3/5
 - 565s - loss: 0.1685 - acc: 0.9346 - acc_top2: 0.9952 - val_loss: 0.5175 - val_acc: 0.8348 - val_acc_top2: 0.9789

Epoch 00003: val_acc did not improve from 0.84976
Epoch 4/5
 - 564s - loss: 0.0917 - acc: 0.9662 - acc_top2: 0.9992 - val_loss: 0.6577 - val_acc: 0.8382 - val_acc_top2: 0.9810

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00004: val_acc did not improve from 0.84976
Epoch 5/5
 - 563s - loss: 0.0422 - acc: 0.9847 - acc_top2: 0.9997 - val_loss: 0.7062 - val_acc: 0.8416 - val_acc_top2: 0.9905

Epoch 00005: val_acc did not improve from 0.84976
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 578s - loss: 0.5005 - acc: 0.7891 - acc_top2: 0.9772 - val_loss: 0.4356 - val_acc: 0.8436 - val_acc_top2: 0.9714

Epoch 00001: val_acc improved from -inf to 0.84364, saving model to ./model/3.hdf5
Epoch 2/5
 - 563s - loss: 0.2890 - acc: 0.8853 - acc_top2: 0.9927 - val_loss: 0.4187 - val_acc: 0.8559 - val_acc_top2: 0.9837

Epoch 00002: val_acc improved from 0.84364 to 0.85588, saving model to ./model/3.hdf5
Epoch 3/5
 - 563s - loss: 0.1610 - acc: 0.9356 - acc_top2: 0.9978 - val_loss: 0.4878 - val_acc: 0.8232 - val_acc_top2: 0.9769

Epoch 00003: val_acc did not improve from 0.85588
Epoch 4/5
 - 563s - loss: 0.0863 - acc: 0.9691 - acc_top2: 0.9993 - val_loss: 0.5408 - val_acc: 0.8491 - val_acc_top2: 0.9810

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00004: val_acc did not improve from 0.85588
Epoch 5/5
 - 564s - loss: 0.0340 - acc: 0.9888 - acc_top2: 1.0000 - val_loss: 0.7314 - val_acc: 0.8518 - val_acc_top2: 0.9748

Epoch 00005: val_acc did not improve from 0.85588
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 579s - loss: 0.5056 - acc: 0.7969 - acc_top2: 0.9701 - val_loss: 0.4071 - val_acc: 0.8366 - val_acc_top2: 0.9837

Epoch 00001: val_acc improved from -inf to 0.83662, saving model to ./model/4.hdf5
Epoch 2/5
 - 564s - loss: 0.2923 - acc: 0.8830 - acc_top2: 0.9912 - val_loss: 0.4136 - val_acc: 0.8523 - val_acc_top2: 0.9850

Epoch 00002: val_acc improved from 0.83662 to 0.85228, saving model to ./model/4.hdf5
Epoch 3/5
 - 563s - loss: 0.1594 - acc: 0.9407 - acc_top2: 0.9964 - val_loss: 0.5621 - val_acc: 0.8373 - val_acc_top2: 0.9871

Epoch 00003: val_acc did not improve from 0.85228
Epoch 4/5
 - 564s - loss: 0.0934 - acc: 0.9677 - acc_top2: 0.9992 - val_loss: 0.7196 - val_acc: 0.8033 - val_acc_top2: 0.9769

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00004: val_acc did not improve from 0.85228
Epoch 5/5
 - 563s - loss: 0.0402 - acc: 0.9873 - acc_top2: 0.9995 - val_loss: 0.7188 - val_acc: 0.8353 - val_acc_top2: 0.9748

Epoch 00005: val_acc did not improve from 0.85228
"""


"""
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 576s - loss: 0.5580 - acc: 0.7659 - acc_top2: 0.9643 - val_loss: 0.6431 - val_acc: 0.7269 - val_acc_top2: 0.9592

Epoch 00001: val_acc improved from -inf to 0.72690, saving model to ./model/0.hdf5
Epoch 2/5
 - 562s - loss: 0.3989 - acc: 0.8374 - acc_top2: 0.9813 - val_loss: 0.3850 - val_acc: 0.8370 - val_acc_top2: 0.9837

Epoch 00002: val_acc improved from 0.72690 to 0.83696, saving model to ./model/0.hdf5
Epoch 3/5
 - 564s - loss: 0.2777 - acc: 0.8844 - acc_top2: 0.9908 - val_loss: 0.4051 - val_acc: 0.8438 - val_acc_top2: 0.9885

Epoch 00003: val_acc improved from 0.83696 to 0.84375, saving model to ./model/0.hdf5
Epoch 4/5
 - 564s - loss: 0.1719 - acc: 0.9296 - acc_top2: 0.9969 - val_loss: 0.4560 - val_acc: 0.8424 - val_acc_top2: 0.9823

Epoch 00004: val_acc did not improve from 0.84375
Epoch 5/5
 - 564s - loss: 0.1056 - acc: 0.9602 - acc_top2: 0.9976 - val_loss: 0.5235 - val_acc: 0.8499 - val_acc_top2: 0.9837

Epoch 00005: val_acc improved from 0.84375 to 0.84986, saving model to ./model/0.hdf5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 581s - loss: 0.5434 - acc: 0.7759 - acc_top2: 0.9703 - val_loss: 0.4984 - val_acc: 0.8077 - val_acc_top2: 0.9857

Epoch 00001: val_acc improved from -inf to 0.80774, saving model to ./model/1.hdf5
Epoch 2/5
 - 567s - loss: 0.3506 - acc: 0.8591 - acc_top2: 0.9883 - val_loss: 0.4105 - val_acc: 0.8281 - val_acc_top2: 0.9878

Epoch 00002: val_acc improved from 0.80774 to 0.82812, saving model to ./model/1.hdf5
Epoch 3/5
 - 566s - loss: 0.2453 - acc: 0.8997 - acc_top2: 0.9942 - val_loss: 0.4283 - val_acc: 0.8424 - val_acc_top2: 0.9885

Epoch 00003: val_acc improved from 0.82812 to 0.84239, saving model to ./model/1.hdf5
Epoch 4/5
 - 567s - loss: 0.1776 - acc: 0.9320 - acc_top2: 0.9978 - val_loss: 0.5334 - val_acc: 0.8342 - val_acc_top2: 0.9837

Epoch 00004: val_acc did not improve from 0.84239
Epoch 5/5
 - 567s - loss: 0.1219 - acc: 0.9550 - acc_top2: 0.9990 - val_loss: 0.5724 - val_acc: 0.8288 - val_acc_top2: 0.9864

Epoch 00005: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00005: val_acc did not improve from 0.84239
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 581s - loss: 0.5399 - acc: 0.7736 - acc_top2: 0.9648 - val_loss: 0.4945 - val_acc: 0.8029 - val_acc_top2: 0.9837

Epoch 00001: val_acc improved from -inf to 0.80286, saving model to ./model/2.hdf5
Epoch 2/5
 - 566s - loss: 0.3471 - acc: 0.8554 - acc_top2: 0.9883 - val_loss: 0.4578 - val_acc: 0.8239 - val_acc_top2: 0.9803

Epoch 00002: val_acc improved from 0.80286 to 0.82393, saving model to ./model/2.hdf5
Epoch 3/5
 - 566s - loss: 0.2638 - acc: 0.8923 - acc_top2: 0.9912 - val_loss: 0.4759 - val_acc: 0.8253 - val_acc_top2: 0.9871

Epoch 00003: val_acc improved from 0.82393 to 0.82529, saving model to ./model/2.hdf5
Epoch 4/5
 - 565s - loss: 0.1807 - acc: 0.9307 - acc_top2: 0.9976 - val_loss: 0.6623 - val_acc: 0.8266 - val_acc_top2: 0.9796

Epoch 00004: val_acc improved from 0.82529 to 0.82665, saving model to ./model/2.hdf5
Epoch 5/5
 - 565s - loss: 0.1335 - acc: 0.9509 - acc_top2: 0.9975 - val_loss: 0.5412 - val_acc: 0.8375 - val_acc_top2: 0.9850

Epoch 00005: val_acc improved from 0.82665 to 0.83753, saving model to ./model/2.hdf5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 581s - loss: 0.5773 - acc: 0.7600 - acc_top2: 0.9658 - val_loss: 0.4447 - val_acc: 0.8212 - val_acc_top2: 0.9701

Epoch 00001: val_acc improved from -inf to 0.82121, saving model to ./model/3.hdf5
Epoch 2/5
 - 567s - loss: 0.3971 - acc: 0.8382 - acc_top2: 0.9850 - val_loss: 0.3919 - val_acc: 0.8491 - val_acc_top2: 0.9837

Epoch 00002: val_acc improved from 0.82121 to 0.84908, saving model to ./model/3.hdf5
Epoch 3/5
 - 567s - loss: 0.2720 - acc: 0.8907 - acc_top2: 0.9930 - val_loss: 0.4469 - val_acc: 0.8158 - val_acc_top2: 0.9776

Epoch 00003: val_acc did not improve from 0.84908
Epoch 4/5
 - 567s - loss: 0.1962 - acc: 0.9225 - acc_top2: 0.9976 - val_loss: 0.6291 - val_acc: 0.8280 - val_acc_top2: 0.9769

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00004: val_acc did not improve from 0.84908
Epoch 5/5
 - 566s - loss: 0.0796 - acc: 0.9711 - acc_top2: 0.9992 - val_loss: 0.6483 - val_acc: 0.8389 - val_acc_top2: 0.9748

Epoch 00005: val_acc did not improve from 0.84908
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            2307        lambda_1[0][0]                   
==================================================================================================
Total params: 101,679,363
Trainable params: 101,679,363
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/5
 - 580s - loss: 0.4970 - acc: 0.7991 - acc_top2: 0.9740 - val_loss: 0.4375 - val_acc: 0.8230 - val_acc_top2: 0.9830

Epoch 00001: val_acc improved from -inf to 0.82301, saving model to ./model/4.hdf5
Epoch 2/5
 - 567s - loss: 0.3078 - acc: 0.8787 - acc_top2: 0.9905 - val_loss: 0.4369 - val_acc: 0.8400 - val_acc_top2: 0.9789

Epoch 00002: val_acc improved from 0.82301 to 0.84003, saving model to ./model/4.hdf5
Epoch 3/5
 - 565s - loss: 0.1924 - acc: 0.9227 - acc_top2: 0.9958 - val_loss: 0.5286 - val_acc: 0.8428 - val_acc_top2: 0.9843

Epoch 00003: val_acc improved from 0.84003 to 0.84275, saving model to ./model/4.hdf5
Epoch 4/5
 - 566s - loss: 0.1219 - acc: 0.9550 - acc_top2: 0.9976 - val_loss: 0.5747 - val_acc: 0.8353 - val_acc_top2: 0.9837

Epoch 00004: val_acc did not improve from 0.84275
Epoch 5/5
 - 566s - loss: 0.0802 - acc: 0.9730 - acc_top2: 0.9990 - val_loss: 0.8076 - val_acc: 0.8285 - val_acc_top2: 0.9748

Epoch 00005: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00005: val_acc did not improve from 0.84275
"""